In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np
import cv2
import tarfile
from pathlib import Path
from tqdm import tqdm
import shutil
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers as tfl
from tensorflow.keras.layers import *
from tensorflow_addons.image import transform as H_transform
import matplotlib.pyplot as plt
import os, datetime

In [ ]:
!pwd

In [ ]:
DATA_PATH = '/root/Internship-Valeo/Project/data'
TMPDIR = '/root/Internship-Valeo/Project/tmp'

In [ ]:
from datasets import synthetic_dataset
from datasets.utils import pipeline
from datasets.utils.pipeline import parse_primitives
from datasets.utils import photometric_augmentation as photaug
from models.homographies import (sample_homography, compute_valid_mask,
                                            warp_points, filter_points)
from models.utils import box_nms

In [ ]:
config = {
            'primitives': 'all',
            'truncate': {'draw_ellipses': 0.3, 'draw_stripes': 0.2, 'gaussian_noise': 0.1},
            'validation_size': 500,
            'test_size': -1,
            'on-the-fly': False,
            'cache_in_memory': True,
            'suffix': 'v6',
            'add_augmentation_to_test_set': False, # set to true to evaluate with noise
            'num_parallel_calls': 10,
            'generation': {
                'split_sizes': {'training': 10000, 'validation': 200, 'test': 500},
                'image_size': [960, 1280],
                'random_seed': 0,
                'params': {
                    'generate_background': {
                        'min_kernel_size': 150, 'max_kernel_size': 500,
                        'min_rad_ratio': 0.02, 'max_rad_ratio': 0.031},
                    'draw_stripes': {'transform_params': (0.1, 0.1)},
                    'draw_multiple_polygons': {'kernel_boundaries': (50, 100)}
                },
            },
            'preprocessing': {
#                 'resize': [240, 320],
#                 'blur_size': 11,
                'resize': [120, 160],
                'blur_size': 21,
            },
            'augmentation': {
                'photometric': {
                    'enable': True,
                    'primitives': ['random_brightness', 'random_contrast', 'additive_speckle_noise',
                'additive_gaussian_noise', 'additive_shade', 'motion_blur'],
                    'params': {
                        'random_brightness': {'max_abs_change': 75},
                        'random_contrast': {'strength_range': [0.3, 1.8]},
                        'additive_gaussian_noise': {'stddev_range': [0, 15]},
                        'additive_speckle_noise': {'prob_range': [0, 0.0035]},
                        'additive_shade':{
                            'transparency_range': [-0.5, 0.8],
                            'kernel_size_range': [50, 100]},
                        'motion_blur': {'max_kernel_size': 7}},
                    'random_order': True,
                },
                'homographic': {
                    'enable': True,
                    'params': {
                        'translation': True,
                        'rotation': True,
                        'scaling': True,
                        'perspective': True,
                        'scaling_amplitude': 0.2,
                        'perspective_amplitude_x': 0.2,
                        'perspective_amplitude_y': 0.2,
                        'patch_ratio': 0.8,
                        'max_angle': 1.57,  # 3.14
                        'allow_artifacts': True,
                        'translation_overflow': 0.05,
                    },
                    'valid_border_margin': 2,
                },
            }
    }
drawing_primitives = [
            'draw_lines',
            'draw_polygon',
            'draw_multiple_polygons',
            'draw_ellipses',
            'draw_star',
            'draw_checkerboard',
            'draw_stripes',
            'draw_cube',
            'gaussian_noise'
    ]

In [ ]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# no need to run this again as synthetic shapes have been generated

def dump_primitive_data(primitive, tar_path, config):
    temp_dir = Path(TMPDIR, primitive)

    synthetic_dataset.set_random_state(np.random.RandomState(
                config['generation']['random_seed']))
    for split, size in config['generation']['split_sizes'].items():
        im_dir, pts_dir = [Path(temp_dir, i, split) for i in ['images', 'points']]
        im_dir.mkdir(parents=True, exist_ok=True)
        pts_dir.mkdir(parents=True, exist_ok=True)

        for i in tqdm(range(size), desc=split, leave=False):
            image = synthetic_dataset.generate_background(
                        config['generation']['image_size'],
                        config['generation']['params']['generate_background'])
            points = np.array(getattr(synthetic_dataset, primitive)(
                        image, config['generation']['params'].get(primitive, {})))
            points = np.flip(points, 1)  # reverse convention with opencv

            b = config['preprocessing']['blur_size']
            image = cv2.GaussianBlur(image, (b, b), 0)
            points = (points * np.array(config['preprocessing']['resize'], np.float)
                          / np.array(config['generation']['image_size'], np.float))
            image = cv2.resize(image, tuple(config['preprocessing']['resize'][::-1]),
                                   interpolation=cv2.INTER_LINEAR)

            cv2.imwrite(str(Path(im_dir, '{}.png'.format(i))), image)
            np.save(Path(pts_dir, '{}.npy'.format(i)), points)

    # Pack into a tar file
    tar = tarfile.open(tar_path, mode='w:gz')
    tar.add(temp_dir, arcname=primitive)
    tar.close()
    shutil.rmtree(temp_dir)

In [ ]:
primitives = parse_primitives(config['primitives'], drawing_primitives)
basepath = Path(
                DATA_PATH, 'synthetic_shapes' +
                ('_{}'.format(config['suffix']) if config['suffix'] is not None else ''))
basepath.mkdir(parents=True, exist_ok=True)

splits = {s: {'images': [], 'points': []}
                  for s in ['training', 'validation', 'test']}

In [ ]:
# no need to run this again as synthetic shapes have been generated

# got an error here so reusing the previously dumped primitive data 25th Feb 2022

for primitive in primitives:
    tar_path = Path(basepath, '{}.tar.gz'.format(primitive))
    if not tar_path.exists():
        dump_primitive_data(primitive, tar_path, config)

In [ ]:
for primitive in primitives:
    # Untar locally
    tar_path = Path(basepath, '{}.tar.gz'.format(primitive))
    tar = tarfile.open(tar_path)
    temp_dir = Path(TMPDIR)
    tar.extractall(path=temp_dir)
    tar.close()
    # Gather filenames in all splits, optionally truncate
    truncate = config['truncate'].get(primitive, 1)
    path = Path(temp_dir, primitive)
    for s in splits:
        e = [str(p) for p in Path(path, 'images', s).iterdir()]
        f = [p.replace('images', 'points') for p in e]
        f = [p.replace('.png', '.npy') for p in f]
        splits[s]['images'].extend(e[:int(truncate*len(e))])
        splits[s]['points'].extend(f[:int(truncate*len(f))])

In [ ]:
# Shuffle
for s in splits:
    perm = np.random.RandomState(0).permutation(len(splits[s]['images']))
    for obj in ['images', 'points']:
        splits[s][obj] = np.array(splits[s][obj])[perm].tolist()

In [ ]:
len(splits['training']['images']), len(splits['test']['images']), len(splits['validation']['images'])

In [ ]:
# code for photometric augmentation

primitives_photo = parse_primitives(config['primitives'], photaug.augmentations)

prim_configs = [config['augmentation']['photometric']['params'].get(p, {}) for p in primitives_photo]

indices = tf.range(len(primitives_photo))
if config['augmentation']['photometric']['random_order']:
    indices = tf.random.shuffle(indices)
def photo_aug_step(i, image):
    fn_pairs = [(tf.equal(indices[i], j), lambda p=p, c=c: getattr(photaug, p)(image, **c))
                for j, (p, c) in enumerate(zip(primitives_photo, prim_configs))]
    image = tf.case(fn_pairs)
    return i + 1, image

In [ ]:
def _gen_shape():
    primitives = parse_primitives(config['primitives'], drawing_primitives)
    while True:
        primitive = np.random.choice(primitives)
        image = synthetic_dataset.generate_background(
                        config['generation']['image_size'],
                        **config['generation']['params']['generate_background'])
        points = np.array(getattr(synthetic_dataset, primitive)(
                        image, **config['generation']['params'].get(primitive, {})))
        yield (np.expand_dims(image, axis=-1).astype(np.float32),
                       np.flip(points.astype(np.float32), 1))

In [ ]:
def _read_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_png(image, channels=1)
    return tf.cast(image, tf.float32)

In [ ]:
# read keypoints
def _read_points(filename):
    return np.load(filename).astype(np.float32)

In [ ]:
if config['on-the-fly']:
    data = tf.data.Dataset.from_generator(
                    _gen_shape, (tf.float32, tf.float32),
                    (tf.TensorShape(config['generation']['image_size']+[1]),
                     tf.TensorShape([None, 2])))
    data = data.map(lambda i, c: pipeline.downsample(
                    i, c, **config['preprocessing']))
else:
    # Initialize dataset with file names
    data = tf.data.Dataset.from_tensor_slices(
                    (splits['training']['images'], splits['training']['points']))
    # Read image and point coordinates
    data = data.map(
                    lambda image, points:
                    (_read_image(image), tf.numpy_function(_read_points, [points], tf.float32)))
    data = data.map(lambda image, points: (image, tf.reshape(points, [-1, 2])))

# if split_name == 'validation':
#     data = data.take(config['validation_size'])
# elif split_name == 'test':
#     data = data.take(config['test_size'])

data = data.map(lambda image, kp: {'image': image, 'keypoints': kp})
data = data.map(pipeline.add_dummy_valid_mask)

In [ ]:
if config['augmentation']['photometric']['enable']:
    data = data.map(lambda d: pipeline.photometric_augmentation(d, **config['augmentation']['photometric']))
if config['augmentation']['homographic']['enable']:
    data = data.map(lambda d: pipeline.homographic_augmentation(
                    d, **config['augmentation']['homographic']))

# Convert the point coordinates to a dense keypoint map
data = data.map(pipeline.add_keypoint_map)
data = data.map(lambda d: {**d, 'image': tf.cast(d['image'], tf.float32) / 255.})

In [ ]:
if config['on-the-fly']:
    val_data = tf.data.Dataset.from_generator(
                    _gen_shape, (tf.float32, tf.float32),
                    (tf.TensorShape(config['generation']['image_size']+[1]),
                     tf.TensorShape([None, 2])))
    val_data = val_data.map(lambda i, c: pipeline.downsample(
                    i, c, **config['preprocessing']))
else:
    # Initialize dataset with file names
    val_data = tf.data.Dataset.from_tensor_slices(
                    (splits['validation']['images'], splits['validation']['points']))
    # Read image and point coordinates
    val_data = val_data.map(
                    lambda image, points:
                    (_read_image(image), tf.numpy_function(_read_points, [points], tf.float32)))
    val_data = val_data.map(lambda image, points: (image, tf.reshape(points, [-1, 2])))

val_data = val_data.map(lambda image, kp: {'image': image, 'keypoints': kp})
val_data = val_data.map(pipeline.add_dummy_valid_mask)

# Convert the point coordinates to a dense keypoint map
val_data = val_data.map(pipeline.add_keypoint_map)
val_data = val_data.map(lambda d: {**d, 'image': tf.cast(d['image'], tf.float32) / 255.})

In [ ]:
model_config = {
            'data_format': 'channels_last',
            'grid_size': 8,
            'detection_threshold': 0.001, # 1/65
            'descriptor_size': 256,
            'batch_size': 64,
            'eval_batch_size': 50,
            'epochs': 50,
            'learning_rate': 0.001,
            'kernel_reg': 0.,
            'lambda_d': 250,
            'descriptor_size': 256,
            'positive_margin': 1,
            'negative_margin': 0.2,
            'lambda_loss': 0.0001,
            'nms': 4,
            'top_k': 0,
            'train_iter': 50000,
            'eval_iter': 200,
            'validation_interval': 1000,
            'seed': 0,
            #'save_interval': 5000,
            #'keep_checkpoints': 20
    }

In [ ]:
# class SyntheticShapes(keras.utils.Sequence):
#     def __init__(self, image_files, point_files, is_training, batch_size = model_config['batch_size']):
#         self.__image_files, self.__point_files = image_files, point_files
#         self.__batch_size = batch_size
#         self.__is_training = is_training
#     def __getitem__(self, index):
#         images = []
#         points_maps = []
#         points_list = []
#         homography_list = [] # not returned yet
#         warped_images = []
#         valid_masks = []
#         warped_points_list = []
#         warped_points_maps = []
        
        
#         if (index + 1)*self.__batch_size > len(self.__image_files):
#             self.__batch_size = len(self.__image_files) - index * self.__batch_size
            
#         batch_img_files = self.__image_files[index * self.__batch_size:(index + 1) * self.__batch_size]
#         batch_point_files = self.__point_files[index * self.__batch_size:(index + 1) * self.__batch_size]
        
#         for img_file, point_file, itr in zip(batch_img_files, batch_point_files, range(self.__batch_size)):

#             image = cv2.imread(img_file, 0)
#             image = np.expand_dims(image, axis = 2)
#             image_shape = tf.shape(image)[:2]
#             points = _read_points(point_file)
#             points = np.round(points).astype(int)
            
#             if self.__is_training:
#                 # add photometric_augmentation
#                 _, image = tf.while_loop(lambda i, image: tf.less(i, len(primitives_photo)),
#                                  photo_aug_step, [0, image], parallel_iterations=1)              
                
#                 # add homography
#                 homography = sample_homography(image_shape, config['augmentation']['homographic']['params'])[0]
#                 warped_image = H_transform(image, homography, interpolation='BILINEAR')
#                 valid_mask = compute_valid_mask(image_shape, homography,
#                                          config['augmentation']['homographic']['params']['valid_border_margin'])
#                 warped_points = warp_points(points, homography)
#                 warped_points = filter_points(warped_points, image_shape)
#                 warped_points = np.round(warped_points).astype(int)
                
#                 warped_kp = tf.minimum(warped_points, image_shape-1)
#                 warped_points_map = tf.scatter_nd(warped_kp, tf.ones([tf.shape(warped_kp)[0]], 
#                                                                      dtype=tf.int32), image_shape)
                        
#                 homography_list.append(homography)
#                 warped_image = warped_image / 255.0
#                 warped_images.append(warped_image)
#                 valid_masks.append(valid_mask)
#                 warped_points_list.append(warped_points)
#                 warped_points_maps.append(warped_points_map)

#             kp = tf.minimum(points, image_shape-1)
#             points_map = tf.scatter_nd(kp, tf.ones([tf.shape(kp)[0]], dtype=tf.int32), image_shape)
            
#             image = image / 255.0
#             images.append(image)
#             points_maps.append(points_map)
#             points_list.append(points)
            
            
#         images = np.array(images)
#         points_maps = np.expand_dims(points_maps, axis = 3)
        
#         if self.__is_training:
#             warped_images = np.array(warped_images)
#             warped_points_maps = np.expand_dims(warped_points_maps, axis = 3)
#             valid_masks = np.array(valid_masks)
#             dummy_loss_target = np.zeros(self.__batch_size)

#             return [warped_images, valid_masks, warped_points_maps], [dummy_loss_target]
        
#         else:
#             return images, points_maps   
           
#     def __len__(self):
#         return int(np.ceil(len(self.__image_files) / float(self.__batch_size)))
    
#     def on_epoch_end(self):
#         pass

In [ ]:
def vgg_block(inputs, filters, kernel_size, name, data_format, training=False,
              batch_normalization=True, kernel_reg=0., **params):
    x = tfl.Convolution2D(filters, kernel_size,
                       kernel_regularizer=tf.keras.regularizers.L2(kernel_reg),
                       data_format=data_format, **params)(inputs)
    if batch_normalization:
        x = tfl.BatchNormalization(
                    fused=True,
                    axis=1 if data_format == 'channels_first' else -1)(x)
    return x

In [ ]:
def shared_encoder(inputs, model_config):
    params_conv = {'padding': 'SAME', 'data_format': model_config['data_format'],
                   'batch_normalization': True, 'activation': tf.nn.relu,
                   'kernel_reg': model_config.get('kernel_reg', 0.)}
    params_pool = {'padding': 'SAME', 'data_format': model_config['data_format']}
    cfirst = model_config['data_format'] == 'channels_first'
    cindex = 1 if cfirst else -1  # index of the channel
    pool_size=(2, 2)
    kernel = 3
    # Encoder
    conv1 = vgg_block(inputs, 64, (kernel, kernel), 'conv1_1', **params_conv)
    conv2 = vgg_block(conv1, 64, (kernel, kernel), 'conv1_2', **params_conv)
    pool1 = MaxPooling2D(pool_size, name="block1_pool", **params_pool)(conv2)

    conv3 = vgg_block(pool1, 64, (kernel, kernel), 'conv2_1', **params_conv)
    conv4 = vgg_block(conv3, 64, (kernel, kernel), 'conv2_2', **params_conv)
    pool2 = MaxPooling2D(pool_size, name="block2_pool", **params_pool)(conv4)

    conv5 = vgg_block(pool2, 128, (kernel, kernel), 'conv3_1', **params_conv)
    conv6 = vgg_block(conv5, 128, (kernel, kernel), 'conv3_2', **params_conv)
    pool3 = MaxPooling2D(pool_size, name="block3_pool", **params_pool)(conv6)

    conv7 = vgg_block(pool3, 128, (kernel, kernel), 'conv4_1', **params_conv)
    conv8 = vgg_block(conv7, 128, (kernel, kernel), 'conv4_2', **params_conv)
    return conv8

In [ ]:
def detector_head(inputs, model_config):
    params_conv = {'padding': 'SAME', 'data_format': model_config['data_format'],
                   'batch_normalization': True,
                   'kernel_reg': model_config.get('kernel_reg', 0.)}
    cfirst = model_config['data_format'] == 'channels_first'
    cindex = 1 if cfirst else -1  # index of the channel

    x = vgg_block(inputs, 256, 3, 'conv1',
                      activation=tf.nn.relu, **params_conv)
    x = vgg_block(x, 1+pow(model_config['grid_size'], 2), 1, 'conv2',
                      activation=None, **params_conv)

    prob = tf.nn.softmax(x, axis=cindex)
    # Strip the extra “no interest point” dustbin
    prob = prob[:, :-1, :, :] if cfirst else prob[:, :, :, :-1]
    prob = tf.nn.depth_to_space(
              prob, model_config['grid_size'], data_format='NCHW' if cfirst else 'NHWC')
    prob = tf.squeeze(prob, axis=cindex)
    return {'logits': x, 'prob': prob}

In [ ]:
def detector_loss(keypoint_map, logits, model_config, valid_mask=None):
    if model_config['data_format'] == 'channels_first':
        logits = tf.transpose(logits, [0, 2, 3, 1])
    # Convert the boolean labels to indices including the "no interest point" dustbin
    labels = keypoint_map[..., tf.newaxis]  # for GPU
    labels = tf.cast(labels, tf.float32)
    labels = tf.nn.space_to_depth(labels, model_config['grid_size'])
    shape = tf.concat([tf.shape(labels)[:3], [1]], axis=0)
    labels = tf.concat([2*labels, tf.ones(shape)], 3)
    # Add a small random matrix to randomly break ties in argmax
    labels = tf.argmax(labels + tf.random.uniform(tf.shape(labels), 0, 0.1), axis=3)
    # Mask the pixels if bordering artifacts appear
    valid_mask = tf.ones_like(keypoint_map) if valid_mask is None else valid_mask
    valid_mask = valid_mask[..., tf.newaxis]  # for GPU
    valid_mask = tf.cast(valid_mask, tf.float32)
    valid_mask = tf.nn.space_to_depth(valid_mask, model_config['grid_size'])
    valid_mask = tf.math.reduce_prod(valid_mask, axis=3)  # AND along the channel dim
    valid_mask = tf.cast(valid_mask, tf.int64)
#     labels = labels * valid_mask
#     loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    loss = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits, weights=valid_mask)
    return loss

In [ ]:
def model_metrics(y_true, y_pred, valid_mask):
    if model_config['nms']:
        prob = tf.map_fn(lambda p: box_nms(p, model_config['nms'],
                                               min_prob=model_config['detection_threshold'],
                                               keep_top_k=model_config['top_k']), y_pred)
    pred = tf.cast(tf.greater_equal(prob, model_config['detection_threshold']), tf.float32)
#     pred = tf.expand_dims(pred, axis = 3)
    pred = valid_mask * pred
    labels = y_true
    precision = tf.math.reduce_sum(pred * labels) / tf.math.reduce_sum(pred)
    recall = tf.math.reduce_sum(pred * labels) / tf.math.reduce_sum(labels)
    return {'precision': precision, 'recall': recall}

In [ ]:
class DetectorLossLayer(Layer):
    def __init__(self, name='detector_loss_layer', trainable=False):
        super(DetectorLossLayer, self).__init__(name=name)
        self.loss_fn = detector_loss
        self.metrics_fn = model_metrics
    def call(self, inputs, targets=None, sample_weight=None):
        loss = self.loss_fn(inputs[0], inputs[1]['logits'], model_config = model_config, valid_mask = inputs[2])
        metrics = self.metrics_fn(inputs[0], inputs[1]['prob'], valid_mask = inputs[2])
        self.add_loss(tf.math.reduce_mean(loss))
        self.add_metric(metrics['precision'], name = 'precision')
        self.add_metric(metrics['recall'], name = 'recall')
        return loss
    def compute_output_shape(self, input_shape):
        return [1]

def net(input_shape1, input_shape2, input_shape3):
    images_inputs = Input(shape = input_shape1, name = 'image')
    valid_masks_inputs = Input(shape = input_shape2, name = 'valid_mask')
    warped_points_inputs = Input(shape = input_shape3, name = 'keypoints')
    
    if model_config['data_format'] == 'channels_first':
        images_inputs1 = tf.transpose(images_inputs, [0, 3, 1, 2])
    
    encoder_output = shared_encoder(images_inputs1, model_config=model_config)
    output = detector_head(encoder_output, model_config=model_config)
    
    loss_layer = DetectorLossLayer()([warped_points_inputs, output, valid_masks_inputs])
    
    model = keras.models.Model(inputs = [images_inputs, valid_masks_inputs, warped_points_inputs] , 
                               outputs = [loss_layer])
    return model

In [ ]:
model = net(input_shape1 = (120, 160, 1), input_shape2 = (120, 160), input_shape3 = (120, 160))

In [ ]:
# model.summary()
keras.utils.plot_model(model, to_file="model.png", show_shapes=False, show_dtype=False, show_layer_names=True,
    rankdir="TB", expand_nested=False, dpi=96)

In [ ]:
def dummy_loss(dummy_target, y_pred):
    return tf.squeeze(y_pred)

model.compile(optimizer = keras.optimizers.Adam(learning_rate = model_config['learning_rate']),
              loss = dummy_loss)

In [ ]:
# batch_size = model_config['batch_size']

# train_gen = SyntheticShapes(splits['training']['images'], splits['training']['points'], 
#                                            batch_size = batch_size, is_training = True)

# val_gen = SyntheticShapes(splits['validation']['images'], splits['validation']['points'], 
#                                          batch_size = batch_size, is_training = True)

# train_steps =  len(splits['training']['images'])/batch_size

In [ ]:
data = data.batch(model_config['batch_size'])

In [ ]:
val_data

In [ ]:
val_data = val_data.take(config['validation_size'])
val_data = val_data.batch(model_config['eval_batch_size'])

In [ ]:
history = model.fit(data , validation_data = val_data, #batch_size = model_config['batch_size'], 
                     #steps_per_epoch = 1000, 
                    epochs=model_config['epochs'], callbacks=[tensorboard_callback])

In [ ]:
# model.save('/root/Internship-Valeo/Project/results/checkpoint-magicpoint-120x160-synth-aug-100322')

In [ ]:
# model.load_weights('/root/Internship-Valeo/Project/results/checkpoint-magicpoint-no_aug')

In [ ]:
x, y = train_gen.__getitem__(0)

In [ ]:
x, y = data.take(2)

In [ ]:
plt.imshow(x['image'][0], cmap = 'gray')

In [ ]:
x['image'].shape

In [ ]:
plt.imshow(x['keypoints'][0], cmap = 'gray')

In [ ]:
plt.imshow(x['valid_mask'][0], cmap = 'gray')

In [ ]:
x['keypoints']

In [ ]:
plt.imshow(x[0][0], cmap = 'gray')

In [ ]:
plt.imshow(x[2][0],cmap = 'gray')

In [ ]:
test_gen = SyntheticShapes(splits['test']['images'], splits['test']['points'], 
                           batch_size = model_config['batch_size'], is_training = True)

In [ ]:
x, y = test_gen.__getitem__(1)

In [ ]:
plt.imshow(x[0][0], cmap = 'gray')

In [ ]:
plt.imshow(x[1][0], cmap = 'gray')

In [ ]:
plt.imshow(x[2][0],cmap = 'gray')

In [ ]:
result = model(x)

In [ ]:
tf.math.reduce_mean(result)

In [ ]:
prob = model.get_layer('tf.compat.v1.squeeze').output
m = keras.models.Model(inputs = model.input, outputs = prob)

y_pred = m.predict(x)

if model_config['nms']:
    prob = tf.map_fn(lambda p: box_nms(p, model_config['nms'],
                                               min_prob=model_config['detection_threshold'],
                                               keep_top_k=model_config['top_k']), y_pred)
    pred = tf.cast(tf.greater_equal(prob, model_config['detection_threshold']), tf.float32)
    
# pred = pred * x[1] # multiplied with valid masks

plt.imshow(pred[0], cmap = 'gray')